# Smalls

## Task
https://smalls.notion.site/Marketing-Analyst-Technical-Challenge-Discussion-65e5d85a71224e2fb9d406f4e772ba37

## Questions
1. What major customer segments do you see? How are these segments different? 
2. Which segments are most valuable? 
3. What recommendations for segment-based initiatives do you have based on your analysis? 
4. What data do you wish you had received to make a more informed recommendation?

In [1]:
# hide code
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()">
  <input type="submit" id="toggleButton" value="Show Code">
</form>
''')

In [2]:
import pandas as pd
path=r'C:\a_job\2022\smalls (cat food)\task'

import sys
sys.path.append(path)

import imp
util = imp.load_source('date_time', r'C:\a_job\2022\smalls (cat food)\date_time.py')

import date_time

df_o=pd.read_excel(f"{path}\data\Analyst Exercise - FACT_ORDERS.xlsx")
df_c=pd.read_excel(f"{path}\data\Analyst Exercise - DIM_CUSTOMERS.xlsx")

print(f"Data contains \nCustomer rows: {df_c.shape[0]} \nOrder rows: {df_o.shape[0]}")

Data contains 
Customer rows: 72717 
Order rows: 209217


# Exploration

In [4]:
#---
cid='CUSTOMER_ID'
oid='ORDER_ID'

# ---
c_cat_features=['TRIAL_PLAN', 'GA_SOURCE', 'GA_MEDIUM',
       'CANCELLATION_FLAG','CANCELLATION_REASON',
       'CAT_COUNT']
c_date_features=['TRIAL_WEEK','CANCELLATION_DATE']

# trial week - a date that starts with monday

# ---
o_cat_features=['ORDER_TYPE', 'ORDER_SEQUENCE']
o_date_features=['ORDER_DATE']
o_date_features=['ORDER_SEQUENCE', 'REVENUE', 'MARGIN']

## Preparation

In [6]:
# convert dates
df_o=date_time.date_parsing(df_o,'ORDER_DATE')
df_c=date_time.date_parsing(df_c,'TRIAL_WEEK')
#--------------------------------------

In [7]:
tmp=pd.merge(df_o,df_c, on=cid, how='left')
a=tmp.loc[tmp['ORDER_SEQUENCE']==1,[cid,'ORDER_DATE','TRIAL_WEEK', 'TRIAL_WEEK_WeekDay']]
a['valid']=(a['ORDER_DATE']>=a['TRIAL_WEEK'])*1

# orders without Trial?
print(f"Users that made order without Trial {a.loc[a['valid']==0].shape[0]}")

Users that made order without Trial 378
